In [35]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasRegressor

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [36]:
def getDataset(data, param):
    df = data.dropna(subset=[param]) 
    return df

def mutate_float(individual, low, up, indpb):
    for i, val in enumerate(individual):
        if random.random() < indpb:
            individual[i] = random.uniform(low, up)
    return individual,

def preprocess_table(data_frame, cocktail_length):
    return pd.concat([
        pd.concat([data_frame.loc[i-1:i-1]] * cocktail_length[i])
        for i in range(1, len(cocktail_length) + 1)
    ])

In [37]:
data =pd.read_excel("./Data.xlsx",sheet_name="culture_result")
cocktail_data=pd.read_excel("./Data.xlsx",sheet_name="cocktail_information")

In [38]:
cocktail_label=cocktail_data["cocktail"].unique()
cocktail_label=np.sort(cocktail_label)
cocktail_length={}

for i in cocktail_label:
    cocktail_length[i]=len(cocktail_data[cocktail_data["cocktail"]==i])
    
cocktail_length

{1: 10, 2: 10, 3: 9, 4: 10, 5: 1, 6: 9, 7: 6, 8: 2}

In [39]:
variable = data.columns[2:61]
variable

Index(['Glycine', 'L-Alanine', 'L-Arginine HCl', 'L-Asparagine Monohydrate',
       'L-Aspartic acid', 'L-Cysteine HCl H2O', 'L-Glutamic Acid',
       'L-Glutamine', 'L-Histidine HCl H2O', 'L-Hydroxyproline',
       'L-Isoleucine', 'L-Leucine', 'L-Lysine HCl', 'L-Methionine',
       'L-Phenylalanine', 'L-Proline', 'L-Serine', 'L-Threonine',
       'L-Tryptophan', 'L-Tyrosine disodium salt dihydrate', 'L-Valine',
       'Calcium Chloride ', 'Magnesium Sulfate, Anhydrous',
       'Potassium Chloride', 'Sodium Bicarbonate', 'Sodium Chloride',
       'Disodium phosphate', 'Cupric Sulfate Pentahydrate',
       'Ferrous Sulfate Heptahydrate', 'Zinc Sulfate Heptahydrate', 'Biotin',
       'Choline bitartrate', 'D-Calcium Pantothenate', 'Folic Acid',
       'Niacinamide', 'p-Aminobenzoic Acid', 'Pyridoxal HCl', 'Pyridoxine HCl',
       'Riboflavin', 'Thiamine HCl', 'Vitamin B12', 'i-inositol',
       'Putrescine 2HCl', 'DL-α-Lipoic Acid', 'Linoleic Acid', 'HEPES',
       'D-(+)-Glucose', 'Hypo

In [40]:
param="fold_change"
data= getDataset(data, param)
data=data.sample(frac=1)
data

,Medium No.,Round,Glycine,L-Alanine,L-Arginine HCl,L-Asparagine Monohydrate,L-Aspartic acid,L-Cysteine HCl H2O,L-Glutamic Acid,L-Glutamine,...,Insulin,Transferrin (apo),Ethanolamine,Sodium selenite,Streptomycin Sulfate,commercial_eRDF,lab_made_eRDF,0.01M HCl,cell_concentration,fold_change
361,349,Round 7,0.123805,0.123805,0.495218,0.247609,0.495218,0.247609,0.247609,1.238046,...,3.290704,3.290704,2.113775,2.113775,2.5,242325.00,22491.0,0.294267,246375.0,10.954382
115,103,Initial,1.500000,3.500000,6.000000,3.000000,6.000000,3.000000,3.000000,35.000000,...,3.500000,3.500000,2.500000,3.500000,2.5,218997.00,18720.0,5.000000,11664.0,0.623077
71,59,Initial,1.500000,1.500000,6.000000,3.000000,6.000000,3.000000,3.000000,15.000000,...,2.500000,2.500000,2.500000,2.500000,2.5,187110.00,6912.0,5.000000,8910.0,1.289062
154,142,GBDT model,1.000000,1.000000,4.000000,2.000000,4.000000,2.000000,2.000000,10.000000,...,3.000000,3.000000,4.000000,4.000000,2.5,206664.75,37557.0,7.000000,83619.0,2.226456
235,223,Round 2,0.672702,0.672702,2.690810,1.345405,2.690810,1.345405,1.345405,6.727024,...,2.975373,2.975373,3.840820,3.840820,2.5,52515.00,20696.0,6.506813,57699.0,2.787930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,220,Round 2,0.968519,0.968519,3.874074,1.937037,3.874074,1.937037,1.937037,9.685186,...,2.373468,2.373468,1.947055,1.947055,2.5,52515.00,20696.0,1.104794,77868.0,3.762466
161,149,GBDT model,1.500000,1.500000,6.000000,3.000000,6.000000,3.000000,3.000000,15.000000,...,3.000000,3.000000,4.000000,4.000000,2.5,206664.75,37557.0,7.000000,55377.0,1.474479
305,293,Round 5,0.704187,0.704187,2.816749,1.408374,2.816749,1.408374,1.408374,7.041872,...,3.302903,3.302903,2.012802,2.012802,2.5,195831.00,232497.0,0.689194,170964.0,0.735339
135,123,Initial,2.500000,2.500000,10.000000,5.000000,10.000000,5.000000,5.000000,25.000000,...,2.500000,2.500000,1.000000,1.000000,2.5,203337.00,10935.0,3.000000,8370.0,0.765432


In [41]:
commercial_average = np.mean(data["commercial_eRDF"])
lab_made_average = np.mean(data["lab_made_eRDF"])

In [79]:
param_grid_svr={"C":[2**-5,2**-3,2**-1,2**1,2**3,2**5,2**7,2**9,2**11],
                "gamma":[2**-20,2**-18,2**-16,2**-14,2**-12,2**-10,2**-8,2**-6,2**-4,2**-2,2**0,2**2,2**4,2**6,2**8,2**10],
                "kernel":["rbf"],
                  "epsilon":[2**-10,2**-8,2**-6,2**-4,2**-2,2**0]}

param_grid_gbdt = {"learning_rate":[i/100 for i in range(1,51,1)],  
                   "max_depth":[2,3,4,5],
                   "n_estimators":[300]}

param_grid_knn={"n_neighbors":[1,2,3,4]}

param_grid_random={"random_state":[0],
                  "max_depth":[2,3,4],
                  "n_estimators":[300]}

def NN_model():
    
    model = Sequential()
    
    model.add(Dense(8, activation='relu', input_dim=len(variable)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='relu'))
    
    model.compile(optimizer=Adam(learning_rate=0.0006),
              loss='mean_squared_error',
              metrics=['accuracy'])
    
    return model

standard = StandardScaler()
minmax = MinMaxScaler()

data_scaled=data[variable].copy()
data_scaled[data_scaled.columns] = standard.fit_transform(data_scaled)
data_scaled[data_scaled.columns] = minmax.fit_transform(data_scaled)
data_scaled[param]=data[param]
    
mlp = KerasRegressor(model=NN_model, batch_size=16, epochs=100,verbose=0)
gbdt=GridSearchCV(GradientBoostingRegressor(),param_grid_gbdt,cv=5)
svr=GridSearchCV(SVR(),param_grid_svr,cv=5)
knn=GridSearchCV(KNeighborsRegressor(),param_grid_knn,cv=5)
    
gbdt.fit(data_scaled[variable] , data_scaled[param])
svr.fit(data_scaled[variable] , data_scaled[param])
knn.fit(data_scaled[variable] , data_scaled[param])

KeyboardInterrupt: 

In [42]:
param_grid_svr={}

param_grid_gbdt = {}

param_grid_knn={}

param_grid_random={}

def NN_model():
    
    model = Sequential()
    
    model.add(Dense(8, activation='relu', input_dim=len(variable)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='relu'))
    
    model.compile(optimizer=Adam(learning_rate=0.0006),
              loss='mean_squared_error',
              metrics=['accuracy'])
    
    return model


standard = StandardScaler()
minmax = MinMaxScaler()

data_scaled=data[variable].copy()
data_scaled[data_scaled.columns] = standard.fit_transform(data_scaled)
data_scaled[data_scaled.columns] = minmax.fit_transform(data_scaled)
data_scaled[param]=data[param]
    
mlp = KerasRegressor(model=NN_model, batch_size=16, epochs=100,verbose=0)
gbdt=GridSearchCV(GradientBoostingRegressor(),param_grid_gbdt,cv=5)
svr=GridSearchCV(SVR(),param_grid_svr,cv=5)
knn=GridSearchCV(KNeighborsRegressor(),param_grid_knn,cv=5)
    
gbdt.fit(data_scaled[variable] , data_scaled[param])
svr.fit(data_scaled[variable] , data_scaled[param])
knn.fit(data_scaled[variable] , data_scaled[param])


GridSearchCV(cv=5, estimator=KNeighborsRegressor(), param_grid={})

In [43]:
iteration = 1
final_results = pd.DataFrame()

while iteration < 21:
    scaled_data = data[variable].copy()
    scaled_data[scaled_data.columns] = standard.transform(scaled_data)
    scaled_data[scaled_data.columns] = minmax.transform(scaled_data)
    scaled_data["fold_change"] = data["fold_change"]

    gbdt_model = GradientBoostingRegressor(**gbdt.best_params_)
    svr_model = SVR(**svr.best_params_)
    knn_model = KNeighborsRegressor(**knn.best_params_)
    estimators = [("svr", svr_model), ("gbdt", gbdt_model), ("mlp", mlp), ("knn", knn_model)]
    
    stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
    stacking_model.fit(scaled_data[variable], scaled_data[param])

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    def evaluate(individual):
        assembled_table = preprocess_table(pd.DataFrame(individual), cocktail_length)
        sorted_cocktail_data = cocktail_data.sort_values("cocktail")
        component_names = sorted_cocktail_data["components"]
        merged_table = pd.concat([
            sorted_cocktail_data.set_axis(component_names, axis=0),
            assembled_table.set_axis(component_names, axis=0)
        ], axis=1).sort_values("number")
        
        computed_values = pd.DataFrame(
            np.array(merged_table["addition"]).reshape(57, 1) * np.array(merged_table.iloc[:, 4:])
        )
        final_table = computed_values.set_axis(list(merged_table["components"]), axis=0).T
        final_table["commercial_eRDF"] = commercial_average
        final_table["lab_made_eRDF"] = lab_made_average
        
        new_scaled_data = final_table[variable].copy()
        new_scaled_data[new_scaled_data.columns] = standard.transform(new_scaled_data)
        new_scaled_data[new_scaled_data.columns] = minmax.transform(new_scaled_data)
        
        return stacking_model.predict(new_scaled_data),
    
    toolbox = base.Toolbox()
    toolbox.register("attribute", random.uniform, 0.025, 3.0)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, 8)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("select", tools.selTournament, tournsize=5)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", mutate_float, low=0.025, up=3.0, indpb=0.2)
    toolbox.register("evaluate", evaluate)

    random.seed(128)
    NUM_GENERATIONS, POPULATION_SIZE, CROSSOVER_PROB, MUTATION_PROB = 50, 300, 0.7, 0.2
    population = toolbox.population(n=POPULATION_SIZE)
    
    for individual in population:
        individual.fitness.values = toolbox.evaluate(individual)
    
    pareto_front = tools.ParetoFront()
    algorithms.eaSimple(population, toolbox, cxpb=CROSSOVER_PROB, mutpb=MUTATION_PROB, ngen=NUM_GENERATIONS, halloffame=pareto_front)
    
    best_solution = tools.selBest(population, 1)[0]
    final_table = preprocess_table(pd.DataFrame(best_solution), cocktail_length)
    sorted_cocktail_data = cocktail_data.sort_values("cocktail")
    component_names = sorted_cocktail_data["components"]
    merged_table = pd.concat([
        sorted_cocktail_data.set_axis(component_names, axis=0),
        final_table.set_axis(component_names, axis=0)
    ], axis=1).sort_values("number")
    
    computed_values = pd.DataFrame(
        np.array(merged_table["addition"]).reshape(57, 1) * np.array(merged_table.iloc[:, 4:])
    )
    final_table = computed_values.set_axis(list(merged_table["components"]), axis=0).T
    final_table["commercial_eRDF"] = commercial_average
    final_table["lab_made_eRDF"] = lab_made_average
    final_table["predicted"] = best_solution.fitness.values[0][0]
    final_table["fold_change"] = 0
    
    final_results = pd.concat([final_results, final_table])
    data = pd.concat([data, final_table])
    iteration += 1

C:\Users\hashizume\Anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\hashizume\Anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	0     
1  	236   


KeyboardInterrupt: 

In [29]:
final_results = final_results.drop(columns=["commercial_eRDF", "lab_made_eRDF", "fold_change"])
final_results = final_results.reset_index(drop=True)
final_results.to_csv("medium_result.csv")

KeyError: "['commercial_eRDF', 'lab_made_eRDF', 'fold_change'] not found in axis"

In [30]:

final_results

,Glycine,L-Alanine,L-Arginine HCl,L-Asparagine Monohydrate,L-Aspartic acid,L-Cysteine HCl H2O,L-Glutamic Acid,L-Glutamine,L-Histidine HCl H2O,L-Hydroxyproline,...,"L-Glutathione, Reduced",Phenol Red,Sodium Pyruvate,Thymidine,Insulin,Transferrin (apo),Ethanolamine,Sodium selenite,Streptomycin Sulfate,predicted
0,0.766031,0.766031,3.064123,1.532061,3.064123,1.532061,1.532061,7.660306,0.766031,1.721761,...,1.744374,22.490098,1.744374,1.744374,1.721761,1.721761,1.744374,1.744374,5.622525,1.377303
1,1.193402,1.193402,4.773607,2.386804,4.773607,2.386804,2.386804,11.934019,1.193402,6.066376,...,2.419236,10.863716,2.419236,2.419236,6.066376,6.066376,2.419236,2.419236,2.715929,1.664674


In [27]:
final_results.to_csv("medium_result.csv")